# Transfer Learning

## 0. Import

In [1]:
import numpy as np
import sys
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical
from keras import optimizers
import utils
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


## 1. Preprocess Tweets

### 1.1. Recreate vocab

In [ ]:
!ls ../dataset/tweets/tweets_emotion_6/emotion_6-processed.csv

In [14]:
FREQ_DIST_FILE = '../dataset/tweets/tweets_polarity_2/tweets_pos_neg_train-processed-freqdist.pkl'
GLOVE_FILE = '../dataset/embedding/glove-seeds.txt'
dim = 200

In [15]:
# Parameters
np.random.seed(1337)
vocab_size = 90000
max_length = 40

In [16]:
# Recreate the original vocab
vocab = utils.top_n_words(FREQ_DIST_FILE, vocab_size, shift=1)

### 1.2. Preprocess Emotion 6

In [17]:
def get_feature_vector(tweet):
    """
    Generates a feature vector for each tweet where each word is
    represented by integer index based on rank in vocabulary.
    """
    words = tweet.split()
    feature_vector = []
    for i in range(len(words) - 1):
        word = words[i]
        if vocab.get(word) is not None:
            feature_vector.append(vocab.get(word))
    if len(words) >= 1:
        if vocab.get(words[-1]) is not None:
            feature_vector.append(vocab.get(words[-1]))
    return feature_vector


def process_tweets(csv_file, test_file=True):
    """
    Generates training X, y pairs.
    """
    tweets = []
    labels = []
    print('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append(feature_vector)
            else:
                tweets.append(feature_vector)
                labels.append(int(sentiment))
            utils.write_status(i + 1, total)
    print('\n')
    return tweets, np.array(labels)

In [6]:
TRAIN_EMOTION_6 = '../dataset/tweets/tweets_emotion_6/emotion_6-processed.csv'

In [7]:
tweets_emo_6, labels_emo_6 = process_tweets(TRAIN_EMOTION_6, test_file=False)

Generating feature vectors
Processing 383456/416809rocessing 63344/416809Processing 159124/416809Processing 161323/416809Processing 184718/416809Processing 226293/416809Processing 276463/416809Processing 309057/416809Processing 325654/416809Processing 374031/416809

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 387310/416809

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 398018/416809

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 409334/416809

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 411699/416809

In [8]:
labels_emo_6 = to_categorical(labels_emo_6)

In [10]:
tweets_emo_6 = pad_sequences(tweets_emo_6, maxlen=max_length, padding='post')

In [11]:
shuffled_indices = np.random.permutation(tweets_emo_6.shape[0])
tweets_emo_6 = tweets_emo_6[shuffled_indices]
labels_emo_6 = labels_emo_6[shuffled_indices]

### 1.3. Preprocess Emotion 4

In [18]:
TRAIN_EMOTION_4 = '../dataset/tweets/tweets_emotion_4/tweets_emotions_train-processed.csv'

In [19]:
tweets_emo_4, labels_emo_4 = process_tweets(TRAIN_EMOTION_4, test_file=False)

Generating feature vectors
Processing 14182/30160

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 17998/30160

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 22031/30160

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 26083/30160

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Processing 30068/30160

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
labels_emo_4 = to_categorical(labels_emo_4)

In [21]:
tweets_emo_4 = pad_sequences(tweets_emo_4, maxlen=max_length, padding='post')

In [22]:
shuffled_indices = np.random.permutation(tweets_emo_4.shape[0])
tweets_emo_4 = tweets_emo_4[shuffled_indices]
labels_emo_4 = labels_emo_4[shuffled_indices]

## 2. Model performance

In [5]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## 3. Transfer learning - (polarity_2 to emotion_6)

### 3.1. Load model (positive/negative CNN)

In [12]:
# Load model
PATH_MODEL_POS_NEG = '../models/base_model/4cnn-04-0.342-0.851-0.389-0.828.hdf5'
transfer_model = load_model(PATH_MODEL_POS_NEG)

In [13]:
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 200)           18000200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 600)           360600    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 300)           540300    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 34, 150)           135150    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 32, 75)            33825     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2400)              0         
__________

In [14]:
# Cut last layers
transfer_model.pop()
transfer_model.pop()

In [15]:
# Freeze layers
for layer in transfer_model.layers:
    layer.trainable = False

In [16]:
# Add news layers
transfer_model.add(Dense(400, name='dense_2'))
transfer_model.add(Activation('relu', name='activation_2'))

transfer_model.add(Dense(6, name='dense_3'))
transfer_model.add(Activation('softmax', name='activation_3'))

### 3.2. Fine tunning and train

In [17]:
# Compile
#adam = optimizers.Adam(lr=5)
transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 200)           18000200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 600)           360600    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 300)           540300    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 34, 150)           135150    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 32, 75)            33825     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2400)              0         
__________

In [ ]:
!ls ./transfer_pol2_emo6/entire_corpus/

In [19]:
# Checkpoints
filepath = "./transfer_pol2_emo6/entire_corpus/{epoch:02d}-{loss:0.3f}-{acc:0.3f}-{val_loss:0.3f}-{val_acc:0.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)

In [20]:
# Fit model
transfer_model.fit(tweets_emo_6, labels_emo_6, batch_size=128, epochs=20, validation_split=0.1, shuffle=True, callbacks=[checkpoint, reduce_lr])

Train on 375128 samples, validate on 41681 samples
Epoch 1/20
375128/375128 [==============================] - 388s 1ms/step - loss: 1.3640 - acc: 0.5028 - val_loss: 1.3349 - val_acc: 0.5098

Epoch 00001: loss improved from inf to 1.36396, saving model to ./transfer_pol2_emo6/entire_corpus/01-1.364-0.503-1.335-0.510.hdf5
Epoch 2/20
375128/375128 [==============================] - 353s 941us/step - loss: 1.3475 - acc: 0.5069 - val_loss: 1.3285 - val_acc: 0.5113

Epoch 00002: loss improved from 1.36396 to 1.34755, saving model to ./transfer_pol2_emo6/entire_corpus/02-1.348-0.507-1.328-0.511.hdf5
Epoch 3/20
375128/375128 [==============================] - 349s 930us/step - loss: 1.3438 - acc: 0.5078 - val_loss: 1.3226 - val_acc: 0.5119

Epoch 00003: loss improved from 1.34755 to 1.34383, saving model to ./transfer_pol2_emo6/entire_corpus/03-1.344-0.508-1.323-0.512.hdf5
Epoch 4/20
375128/375128 [==============================] - 353s 942us/step - loss: 1.3408 - acc: 0.5087 - val_loss: 1.31

## 4. Transfer learning - (emotion_6 to emotion_4)

### 4.1. Load model (positive/negative CNN)

In [ ]:
!ls ../models/transfer_pol2_emo6/entire_corpus/20-1.326-0.513-1.307-0.517.hdf5

In [31]:
# Load model
PATH_MODEL_EMO_6 = '../models/transfer_pol2_emo6/entire_corpus/20-1.326-0.513-1.307-0.517.hdf5'
transfer_model = load_model(PATH_MODEL_EMO_6)

In [4]:
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 200)           18000200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 600)           360600    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 300)           540300    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 34, 150)           135150    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 32, 75)            33825     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2400)              0         
__________

In [32]:
# Cut last layers
transfer_model.pop()
transfer_model.pop()

In [33]:
# Freeze layers
for layer in transfer_model.layers:
    layer.trainable = False

In [34]:
# Add news layers
transfer_model.add(Dense(350, name='dense_4'))
transfer_model.add(Activation('relu', name='activation_4'))

transfer_model.add(Dense(4, name='dense_5'))
transfer_model.add(Activation('softmax', name='activation_5'))

### 4.2. Fine tunning and train

In [35]:
# Compile
transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])

In [10]:
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 200)           18000200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 600)           360600    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 300)           540300    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 34, 150)           135150    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 32, 75)            33825     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2400)              0         
__________

In [ ]:
!ls ../models/transfer_emo6_emo4/f1/

In [36]:
# Checkpoints
filepath = "../models/transfer_emo6_emo4/f1/{epoch:02d}-{loss:0.3f}-{val_f1:0.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)

In [37]:
# Fit model
transfer_model.fit(tweets_emo_4, labels_emo_4, batch_size=128, epochs=20, validation_split=0.1, shuffle=True, callbacks=[checkpoint, reduce_lr])

Train on 27144 samples, validate on 3016 samples
Epoch 1/20
27144/27144 [==============================] - 27s 990us/step - loss: 0.9760 - f1: 0.5299 - val_loss: 0.9430 - val_f1: 0.5609

Epoch 00001: loss improved from inf to 0.97604, saving model to ../models/transfer_emo6_emo4/f1/01-0.976-0.561.hdf5
Epoch 2/20
27144/27144 [==============================] - 27s 992us/step - loss: 0.9545 - f1: 0.5427 - val_loss: 0.9430 - val_f1: 0.5573

Epoch 00002: loss improved from 0.97604 to 0.95450, saving model to ../models/transfer_emo6_emo4/f1/02-0.954-0.557.hdf5
Epoch 3/20
27144/27144 [==============================] - 27s 992us/step - loss: 0.9452 - f1: 0.5498 - val_loss: 0.9271 - val_f1: 0.5533

Epoch 00003: loss improved from 0.95450 to 0.94521, saving model to ../models/transfer_emo6_emo4/f1/03-0.945-0.553.hdf5
Epoch 4/20
27144/27144 [==============================] - 27s 997us/step - loss: 0.9463 - f1: 0.5464 - val_loss: 0.9319 - val_f1: 0.5618

Epoch 00004: loss did not improve from 0.94

## 5. Final model testing

In [3]:
import pandas as pd

In [ ]:
!ls ../models/transfer_emo6_emo4/entire_corpus/20-0.921-0.608-0.937-0.592.hdf5

In [13]:
# Load model
PATH_MODEL_EMO_4 = '../models/transfer_emo6_emo4/entire_corpus/20-0.921-0.608-0.937-0.592.hdf5'
transfer_model = load_model(PATH_MODEL_EMO_4)

In [14]:
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 200)           18000200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 600)           360600    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 300)           540300    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 34, 150)           135150    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 32, 75)            33825     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2400)              0         
__________

In [ ]:
!ls ../dataset/tweets/tweets_emotion_4/tweets_emotions_prediction-processed.csv

In [4]:
TEST_PROCESSED_FILE = '../dataset/tweets/tweets_emotion_4/tweets_emotions_prediction-processed.csv'

In [10]:
test_tweets, _ = process_tweets(TEST_PROCESSED_FILE, test_file=True)

Generating feature vectors
Processing 2755/2755



In [11]:
test_tweets = pad_sequences(test_tweets, maxlen=max_length, padding='post')

In [15]:
predictions = transfer_model.predict(test_tweets, batch_size=128, verbose=1)

2755/2755 [==============================] - 2s 622us/step


In [ ]:
results = zip(map(str, range(len(test_tweets))), np.round(predictions[:, 0]).astype(int))
utils.save_results_to_csv(results, '.csv')

In [17]:
predictions

array([[0.36825687, 0.04995044, 0.5345072 , 0.04728551],
       [0.02165285, 0.00963362, 0.75461566, 0.21409786],
       [0.00128797, 0.0012112 , 0.436287  , 0.56121385],
       ...,
       [0.46889895, 0.263454  , 0.21271881, 0.05492825],
       [0.14677207, 0.06191447, 0.69982505, 0.09148838],
       [0.01749193, 0.00645487, 0.5592364 , 0.4168168 ]], dtype=float32)

In [16]:
conclusions = ['angry', 'sad', 'others', 'happy']

In [ ]:
!ls ../dataset/tweets/tweets_emotion_4/tweets_emotions_prediction.txt

In [18]:
TEST_FILE_INPUT = '../dataset/tweets/tweets_emotion_4/tweets_emotions_prediction.txt'

In [27]:
test_df = pd.read_csv(TEST_FILE_INPUT, sep='\t')

In [86]:
for i in range(4):
    test_df[conclusions[i]] = predictions.T[i]

In [87]:
test_df.head()

,id,turn1,turn2,turn3,angry,sad,others,happy
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,0.368257,0.049950,0.534507,0.047286
1,1,Mixed things such as??,the things you do.,Have you seen minions??,0.021653,0.009634,0.754616,0.214098
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,0.001288,0.001211,0.436287,0.561214
3,3,Woah bring me some,left it there oops,Brb,0.125436,0.055360,0.559979,0.259225
4,4,it is thooooo,I said soon master.,he is pressuring me,0.306928,0.166277,0.488765,0.038030


In [88]:
TEST_FILE_OUTPUT = '../dataset/tweets/tweets_emotion_4/tweets_emotions_prediction_probabilities.txt'

In [91]:
test_df.to_csv(TEST_FILE_OUTPUT, sep='\t', index=None)

In [20]:
df_predict = [conclusions[np.argmax(predictions[i])] for i in range(len(predictions))]

In [21]:
df_predict

['others',
 'others',
 'happy',
 'others',
 'others',
 'others',
 'sad',
 'others',
 'others',
 'angry',
 'others',
 'others',
 'angry',
 'angry',
 'others',
 'others',
 'others',
 'others',
 'angry',
 'others',
 'others',
 'sad',
 'others',
 'sad',
 'others',
 'angry',
 'others',
 'sad',
 'sad',
 'others',
 'others',
 'angry',
 'others',
 'others',
 'happy',
 'others',
 'angry',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'angry',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'angry',
 'others',
 'others',
 'others',
 'angry',
 'angry',
 'angry',
 'others',
 'others',
 'others',
 'angry',
 'others',
 'others',
 'others',
 'sad',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'happy',
 'others',
 'others',
 'others',
 'angry',
 'angry',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 'others',
 '

In [24]:
test_df.head(5)

,id,turn1,turn2,turn3,label
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,others
1,1,Mixed things such as??,the things you do.,Have you seen minions??,others
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,happy
3,3,Woah bring me some,left it there oops,Brb,others
4,4,it is thooooo,I said soon master.,he is pressuring me,others


In [23]:
test_df['label'] = df_predict

In [25]:
TEST_FILE_OUTPUT = '../results/tweets_emotions_predictions.txt'

In [26]:
test_df.to_csv(TEST_FILE_OUTPUT, sep='\t', index=None)